In [ ]:
import openvino as ov
model = ov.convert_model('/home/sr/ov_fr/buffalo_l/det_10g.onnx', example_input=(1, 3, 640, 640))

In [2]:
ov.serialize(model, xml_path="/home/sr/ov_fr/buffalo_l/det_10g_640.xml", bin_path="/home/sr/ov_fr/buffalo_l/det_10g_640.bin")

In [2]:
import openvino as ov

# 1. Convert your ONNX model to OpenVINO in-memory model (still FP32)
model = ov.convert_model("/home/sr/ov_fr/buffalo_l/adaface_ir_50_model.onnx")

# 2. Save the IR model to disk, auto-generating both .xml and .bin in FP16
ov.save_model(
    model,
    "/home/sr/ov_fr/buffalo_l/adaface_ir_50_model_fp16.xml",
    compress_to_fp16=True
)

In [ ]:
import onnx 
model = onnx.load('/home/sr/ov_fr/buffalo_l/adaface_ir_50_model.onnx')

In [ ]:
import os
import cv2
import numpy as np
from nncf import Dataset
from typing import List

def preprocess_image(image_path: str) -> np.ndarray:
    """Preprocess a single image for AdaFace (112x112, normalized)."""
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Failed to load image: {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    
    # Resize to 112x112 (AdaFace input size)
    img = cv2.resize(img, (112, 112))
    
    # Normalize to [0, 1]
    img = img.astype(np.float32) / 255.0
    
    # Apply mean and std normalization (adjust as per AdaFace requirements)
    mean = np.array([0.5, 0.5, 0.5])
    std = np.array([0.5, 0.5, 0.5])
    img = (img - mean) / std
    
    # Convert to model input format (1, C, H, W)
    img = img.transpose(2, 0, 1)  # HWC to CHW
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

def load_images(image_dir: str, max_images: int = 2000) -> List[np.ndarray]:
    """Load and preprocess images from directory."""
    image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
    if not image_paths:
        raise ValueError(f"No images found in directory: {image_dir}")
    image_paths = image_paths[:max_images]  # Limit to max_images
    images = []
    for path in image_paths:
        try:
            images.append(preprocess_image(path))
        except Exception as e:
            print(f"Skipping image {path}: {e}")
    return images

def create_calibration_dataset(image_dir: str, max_images: int = 2000) -> Dataset:
    """Create NNCF calibration dataset."""
    images = load_images(image_dir, max_images)
    if not images:
        raise ValueError("No valid images loaded for calibration dataset")
    def transform_fn(image):
        return {"input": image}  # Use 'input' to match the model's input name
    return Dataset(images, transform_fn)

# Example usage
if __name__ == "__main__":
    image_dir = "/home/sr/calib-data" # Path to your folder with 4000 images
    calibration_dataset = create_calibration_dataset(image_dir, max_images=2000)
    print(f"Calibration dataset created with {len(calibration_dataset.get_data())} samples")

In [8]:
import onnx
import nncf
import openvino as ov
from onnx import helper

# Load the ONNX model
onnx_model_path = "/home/sr/ov_fr/buffalo_l/adaface_ir_50_model.onnx"  # Replace with your model path
onnx_model = onnx.load(onnx_model_path)

# Verify model input name
model_inputs = [input.name for input in onnx_model.graph.input]
print(f"Model input names: {model_inputs}")  # Should include 'input'

# Create calibration dataset
image_dir = "/home/sr/calib-data"  # Path to your folder with 4000 images
calibration_dataset = create_calibration_dataset(image_dir, max_images=2000)

# Verify calibration dataset input
sample_data = next(iter(calibration_dataset.get_inference_data()))
print(f"Calibration dataset input keys: {list(sample_data.keys())}")  # Should include 'input'

# Quantize the model to INT-8
quantized_model = nncf.quantize(
    onnx_model,
    calibration_dataset,
    fast_bias_correction=False,  # Improve accuracy
    subset_size=2000,  # Use 2000 images for calibration
    target_device=nncf.TargetDevice.CPU  # Optimize for CPU
)

# Save the quantized ONNX model
quantized_model_path = "quantized_adaface.onnx"
onnx.save(quantized_model, quantized_model_path)

# Convert to OpenVINO IR
ov_model = ov.convert_model(quantized_model_path)

# Save the OpenVINO IR model
ov.save_model(ov_model, "adaface_int8.xml")

Model input names: ['input']
Calibration dataset input keys: ['data']


ValueError: Required inputs (['input']) are missing from input feed (['data']).